# 📊 Análisis exploratorio: 🏦 Bank Marketing Dataset (UCI)

## 1️⃣ **Introducción y contexto del proyecto**

El análisis exploratorio que se va a realizar está basado en el dataset **Bank Marketing**, publicado por el Banco de Portugal y disponible en el repositorio UCI Machine Learning:  
🔗 https://archive.ics.uci.edu/dataset/222/bank+marketing  

Este conjunto de datos recoge información de campañas de marketing telefónico realizadas por una entidad bancaria portuguesa. El banco contactaba a clientes potenciales con el fin de ofrecerles un depósito a plazo fijo. Cada fila representa a un cliente contactado y las variables recogen aspectos demográficos, financieros y operativos del proceso de captación. Los datos recogidos en este dataset están ordenadores por fecha, desde Mayo del 2008 a Noviembre del 2010.

El fichero que analizaremos, **bank-full.csv**, contiene **45.211 registros** y **17 variables** relacionadas con datos demográficos, laborales, financieros y características de la interacción comercial.

Aunque el objetivo original del dataset era predecir si un cliente contrataría un depósito a plazo fijo, en este proyecto vamos a reutilizar estas variables para analizar y modelar dos grandes escenarios reales dentro del sector financiero: **gestión de patrimonio** y **gestión del riesgo**.

---

## 🎯 **Objetivo del análisis**

El propósito de este análisis es **comprender el perfil financiero y sociodemográfico de los clientes del banco**, analizando sus patrones económicos y su posible estabilidad o capacidad de ahorro.

Para ello, exploraremos en profundidad la columna `balance`, que refleja el **saldo anual promedio de la cuenta bancaria del cliente**. A partir de esta métrica construiremos **dos posibles variables objetivo (targets)**, inspiradas en escenarios reales del sector bancario:

---


## 🅰️ **Escenario A: El Umbral del 90% — Gestión de Patrimonio**

En banca privada existe un fenómeno conocido: **el 10% de los clientes más ricos generan alrededor del 80% de los ingresos del banco** (Principio de Pareto). Identificar este grupo es clave para priorizar servicios personalizados y productos de alto valor.

**Acción realista:**  
Calcular el **percentil 90 (P90)** del `balance`. Ese valor representa el **mínimo saldo que posee el 10% de clientes más adinerados**.

A partir de ese umbral crearemos la variable binaria `Patrimonio_Alto`:

- `Patrimonio_Alto = 1` si `balance > P90`  
- `Patrimonio_Alto = 0` si `balance ≤ P90`

**Interpretación práctica:**  
El modelo resultante permitirá estimar si un cliente **pertenece al segmento de alto patrimonio**, a partir de su edad, trabajo, educación, historial crediticio y comportamiento financiero.  
Esto imita cómo las divisiones de **Wealth Management** detectan clientes con potencial para productos premium.

---

## 🅱️ **Escenario B: El Umbral Cero — Riesgo y Sobregiro**

En gestión de riesgos bancarios, uno de los indicadores más críticos es detectar **clientes con saldo negativo** o riesgo de entrar en sobregiro. Un balance por debajo de cero señala inestabilidad financiera o dependencia del crédito.

**Acción realista:**  
Utilizar el valor **0 como umbral**.

Crearemos la variable binaria `Riesgo_Sobregiro`:

- `Riesgo_Sobregiro = 1` si `balance < 0`  
- `Riesgo_Sobregiro = 0` si `balance ≥ 0`

**Interpretación práctica:**  
El modelo predecirá qué clientes tienen una **alta probabilidad de estar en números rojos**, lo que resulta esencial para los departamentos de **Risk Management**.  
Esto ayuda a anticipar comportamientos financieros que puedan derivar en impagos o necesidad de intervención.

---

## 🧩 **Significado de las variables del dataset**

| **Columna** | **Descripción** |
|-------------|-----------------|
| `age` | Edad del cliente. |
| `job` | Tipo de trabajo u ocupación. |
| `marital` | Estado civil. |
| `education` | Nivel de educación alcanzado. |
| `default` | ¿Tiene impagos en créditos anteriores? |
| `balance` | Saldo medio anual de la cuenta bancaria. |
| `housing` | ¿Tiene hipoteca en curso? (yes/no) |
| `loan` | ¿Tiene algún préstamo personal? |
| `contact` | Tipo de contacto empleado (teléfono móvil o fijo). |
| `day` | Día del mes en que se realizó el último contacto. |
| `month` | Mes del último contacto. |
| `duration` | Duración (en segundos) de la última llamada. |
| `campaign` | Número de contactos realizados durante la campaña. |
| `pdays` | Días desde el último contacto previo a esta campaña. |
| `previous` | Número de contactos en campañas anteriores. |
| `poutcome` | Resultado de campañas anteriores. |
| `y` | Variable original del banco: ¿suscribió el depósito a plazo? (yes/no). |

---

📌 *Fuente de los datos: UCI Machine Learning Repository, Bank Marketing Dataset (2012).*  

Este análisis nos permitirá comprender mejor los perfiles financieros de los clientes y construir modelos predictivos aplicables a contextos reales de banca, tanto en **gestión de patrimonio** como en **evaluación de riesgo**.

---

## 2️⃣ **Importación de librerías y carga del dataset**

Incluye todas las librerías necesarias para manipular datos, visualizar resultados y entrenar modelos.  
Se carga también el fichero `bank-full.csv` y se muestran sus primeras filas para verificar que se ha importado correctamente.

In [1]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np
import io # Agregamos io para el manejo de strings como archivos

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y modelos (scikit-learn)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos que usaremos más adelante
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Métricas
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Cargamos el dataset y visualizamos las primeras filas

In [2]:
dataset = './data/bank-full.csv'

# 1. Leer el archivo completo como texto
try:
    with open(dataset, 'r', encoding='utf-8') as f:
        content = f.read()
except FileNotFoundError:
    # Manejo de error si el archivo no está en la ubicación esperada
    print(f"Error: No se encontró el archivo '{dataset}'. Asegúrate de que está cargado correctamente.")
    exit()

# 2. Modificación solicitada: Reemplazar todas las comillas dobles (") con una cadena vacía
# Esto limpia tanto el encabezado como el resto de los datos.
cleaned_content = content.replace('"', '')

# 3. Leer el DataFrame usando io.StringIO
# io.StringIO permite a Pandas leer la cadena de texto como si fuera un archivo.
df = pd.read_csv(io.StringIO(cleaned_content), sep=';')

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


---

## 3️⃣ **Exploración inicial del dataset (EDA básico)**

Este apartado da las primeras pistas sobre posibles problemas o particularidades del dataset.

- .info(), .describe(), .nunique()
- Dimensiones, tipos de datos, distribución inicial.

In [3]:
# Dimensiones del dataset
df.shape

(45211, 17)

In [4]:
# Información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [5]:
# Estadísticas generales
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [6]:
# Número de valores únicos por columna
df.nunique()

age            77
job            12
marital         3
education       4
default         2
balance      7168
housing         2
loan            2
contact         3
day            31
month          12
duration     1573
campaign       48
pdays         559
previous       41
poutcome        4
y               2
dtype: int64

In [7]:
# Revisamos la distribución inicial de la variable objetivo para saber si el dataset está desbalanceado
df['balance'].value_counts()

balance
 0        3514
 1         195
 2         156
 4         139
 3         134
          ... 
-381         1
 4617        1
 20584       1
 4358        1
 16353       1
Name: count, Length: 7168, dtype: int64

---

## 4️⃣ **Limpieza de datos**
En esta parte se revisa y corrige la calidad de los datos:
- Detección de valores nulos  
- Comprobación de categorías inconsistentes  
- Revisión de posibles valores atípicos extremos que distorsionen el análisis  
El propósito es garantizar que el dataset sea adecuado para construir modelos fiables.

In [8]:
# Revisamos los valores nulos que pueda tener el dataset
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [9]:
print("\n--- CANTIDAD TOTAL DE DUPLICADOS ---")
print(df.duplicated().sum())


--- CANTIDAD TOTAL DE DUPLICADOS ---
0


In [10]:
# Definir el cambio de nombre
rename_map = {
    'y': 'subscribed_term_deposit'
}

# Aplicar el cambio de nombre
df = df.rename(columns=rename_map)

# Mostrar las primeras filas y los nombres de las columnas para verificar
print("Nombres de columnas después de renombrar:")
print(df.columns.tolist())

Nombres de columnas después de renombrar:
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'subscribed_term_deposit']


---

## 5️⃣ **Análisis profundo del balance (variable clave del proyecto)**
La columna `balance` es el pilar de las dos variables objetivo.  
En este apartado se debe:
- Analizar la distribución del balance  
- Identificar valores negativos, picos extraños o colas largas  
- Calcular rangos, percentiles clave (P50, P75, P90, P95)  
- Visualizarlo con histogramas, boxplots o KDE  
Este análisis justifica los umbrales propuestos para los dos escenarios financieros.

---

## 6️⃣ **Creación de los dos escenarios (targets)**

En ambos casos se debe analizar qué proporción de clientes pertenece a cada grupo y si es necesario aplicar técnicas para tratar desbalances.

### 🔹 A: `Patrimonio_Alto`
- Calcular el percentil 90 (`np.percentile(df['balance'], 90)`)  
- Crear la variable binaria según el umbral  
- Comprobar cuántos registros quedan en cada clase (balanceadas/desbalanceadas)

In [11]:
# Suponiendo que tu DataFrame con la columna 'balance' se llama df
# Usaremos el DataFrame df, que ya tiene los datos limpios.

# Calcular el percentil 90 de la columna 'balance'
percentil_90_umbral = np.percentile(df['balance'], 90)

print(f"El valor del Percentil 90 de 'balance' es: {percentil_90_umbral:,.2f}")
print("---------------------------------")

El valor del Percentil 90 de 'balance' es: 3,574.00
---------------------------------


In [12]:
# Crear la variable binaria 'Patrimonio_Alto'
# 1 si el balance es mayor que el umbral, 0 si no lo es.
df['Patrimonio_Alto'] = np.where(df['balance'] > percentil_90_umbral, 1, 0)

# Comprobar la creación de la nueva columna
print("Primeras filas con la nueva variable 'Patrimonio_Alto':")
print(df[['balance', 'Patrimonio_Alto']].head(10))

Primeras filas con la nueva variable 'Patrimonio_Alto':
   balance  Patrimonio_Alto
0     2143                0
1       29                0
2        2                0
3     1506                0
4        1                0
5      231                0
6      447                0
7        2                0
8      121                0
9      593                0


In [13]:
# Contar el número de registros en cada clase
conteo_clases = df['Patrimonio_Alto'].value_counts()
proporcion_clases = df['Patrimonio_Alto'].value_counts(normalize=True) * 100

print("\nResultados del Análisis de Desbalance de 'Patrimonio_Alto':")
print("---------------------------------------------------------")
print(f"Conteo de Clases:\n{conteo_clases}")
print("\nProporción de Clases:")
print(proporcion_clases.map('{:.2f}%'.format))


Resultados del Análisis de Desbalance de 'Patrimonio_Alto':
---------------------------------------------------------
Conteo de Clases:
Patrimonio_Alto
0    40690
1     4521
Name: count, dtype: int64

Proporción de Clases:
Patrimonio_Alto
0    90.00%
1    10.00%
Name: proportion, dtype: object


### 🔹 B: `Riesgo_Sobregiro`
- Aplicar el umbral cero  
- Crear la variable binaria correspondiente  
- Revisar la distribución resultante  

In [14]:
# Definir el umbral como 0
umbral_cero = 0

# Crear la variable binaria 'Riesgo_Sobregiro'
# 1 si el balance es menor que cero (riesgo/sobregiro), 0 si es cero o positivo.
df['Riesgo_Sobregiro'] = np.where(df['balance'] < umbral_cero, 1, 0)

# Comprobar la creación de la nueva columna
print("Primeras filas con la nueva variable 'Riesgo_Sobregiro':")
print(df[['balance', 'Riesgo_Sobregiro']].head(10))

Primeras filas con la nueva variable 'Riesgo_Sobregiro':
   balance  Riesgo_Sobregiro
0     2143                 0
1       29                 0
2        2                 0
3     1506                 0
4        1                 0
5      231                 0
6      447                 0
7        2                 0
8      121                 0
9      593                 0


In [15]:
# Contar el número de registros en cada clase
conteo_clases_riesgo = df['Riesgo_Sobregiro'].value_counts()
proporcion_clases_riesgo = df['Riesgo_Sobregiro'].value_counts(normalize=True) * 100

print("\nResultados del Análisis de Desbalance de 'Riesgo_Sobregiro':")
print("----------------------------------------------------------")
print(f"Conteo de Clases:\n{conteo_clases_riesgo}")
print("\nProporción de Clases:")
print(proporcion_clases_riesgo.map('{:.2f}%'.format))


Resultados del Análisis de Desbalance de 'Riesgo_Sobregiro':
----------------------------------------------------------
Conteo de Clases:
Riesgo_Sobregiro
0    41445
1     3766
Name: count, dtype: int64

Proporción de Clases:
Riesgo_Sobregiro
0    91.67%
1     8.33%
Name: proportion, dtype: object


---

## 7️⃣ **Análisis univariante y bivariante**
En este apartado se exploran relaciones relevantes:
- Distribuciones individuales de variables categóricas y numéricas  
- Relación entre variables demográficas (age, job, education…) y `balance`  
- Relación entre variables y cada target (`Patrimonio_Alto` o `Riesgo_Sobregiro`)  
- Visualizaciones: countplots, barplots, boxplots, heatmaps de correlación  

El objetivo es descubrir patrones que puedan ser útiles para los modelos predictivos.

# Preprocesamiento A

In [16]:
# Separar variables predictoras y target
X_a = df.drop('Patrimonio_Alto', axis=1)
y_a = df['Patrimonio_Alto']

# Identificar columnas para preprocesamiento
num_features = X_a.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X_a.select_dtypes(include=['object']).columns.tolist()

# Pipeline para transformaciones (Escalado para numéricas y One-Hot para categóricas)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough' # Incluye la edad sin transformar (o 'passthrough' si la dejaste)
)

# Creamos el Pipeline Completo con el preprocesador y el modelo
model_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(solver='liblinear', random_state=42))])

---

## 8️⃣ **Preprocesamiento para Machine Learning**
Aquí se preparan los datos antes de entrenar modelos:
- Identificar variables categóricas y numéricas  
- Crear transformadores:  
  - `OneHotEncoder` para categorías  
  - `StandardScaler` para variables numéricas  
- Separar en train/test  

---

## 9️⃣ **Entrenamiento de modelos de clasificación**
Para cada escenario (A y/o B), se entrenan modelos como:
- Regresión Logística  
- Árbol de Decisión  
- Random Forest  
- KNN  
- Naive Bayes  
- SVM  
- MLP (Red Neuronal sencilla)

El enfoque es comparativo: ver cuál funciona mejor según las métricas.

---

## 🔟 **Evaluación de modelos**
Se analizan:
- Accuracy  
- F1-score  
- Matriz de confusión  
- Clasification report  

El objetivo es seleccionar el modelo más robusto según el caso de uso (segmentación de clientes o gestión de riesgo).


---

## 1️⃣1️⃣ **Conclusiones**
La última sección resume:
- Principales hallazgos del EDA  
- Interpretación financiera de los resultados  
- Mejor modelo encontrado y por qué  
- Limitaciones del dataset  
- Posibles mejoras o líneas futuras  